In [12]:
import time
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2

In [15]:
from os.path import exists

def downloadDataset():
  !wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
  !unzip -q tiny-imagenet-200.zip && ls tiny-imagenet-200

is_exists = exists('tiny-imagenet-200.zip')
if not is_exists:
  downloadDataset()

In [16]:
def mapDirectoryToInt():
  ID_DICT = {}
  PATH = 'tiny-imagenet-200/wnids.txt'
  FILEOPEN = open(PATH,'r')
  for i, folder in enumerate(FILEOPEN):
    ID_DICT[folder.replace('\n','')]=i
  return ID_DICT

ID_DICT = mapDirectoryToInt()

In [17]:
def getImage(folderName=None):
    DATASET_PATH = 'tiny-imagenet-200/'+folderName+'/'
    directories = os.listdir(DATASET_PATH)
    ONE_HOT_ENCODED = np.eye(len(directories))
    MAP_DIC = mapDirectoryToInt()
    TRAIN_IMAGES = []
    TRAIN_LABELS = []
    CLASS = None
    for directory in directories:
        path = DATASET_PATH+directory+'/'
        files = os.listdir(path)
        if files[0].endswith('.txt'):
            CLASS = files[0]
        else:
            CLASS = files[1]
        for file in files:
            if file.endswith('.txt'):
                continue
            IMG_FOLDER_PATH = path+file+'/'
            IMAGES = os.listdir(IMG_FOLDER_PATH)
            for IMAGE in IMAGES:
                IMG_PATH = IMG_FOLDER_PATH+IMAGE
                img = cv2.imread(IMG_PATH)
                if len(img.shape)< 3:
                    continue
                #img = np.reshape(img,[64,64,3])
                TRAIN_IMAGES.append(img)
                TRAIN_LABELS.append((np.reshape(ONE_HOT_ENCODED[MAP_DIC[CLASS.replace('_boxes.txt','')]], [200])))
    return np.array(TRAIN_IMAGES), np.array(TRAIN_LABELS)

In [18]:
def loadValidationData(folderName=None):
  DATASET_PATH = 'tiny-imagenet-200/'
  DATASET_FILE_PATH = 'tiny-imagenet-200/'+folderName+'/val_annotations.txt'
  ONE_HOT_ENCODED = np.eye(200)
  MAP_DIC = mapDirectoryToInt()
  file = open(DATASET_FILE_PATH)
  TEST = []
  TEST_LABELS = []
  cnt = 0
  for line in file:
    image, class_id = line.split('\t')[:2]
    IMG_PATH = DATASET_PATH+folderName+'/images/'+image
    img = cv2.imread(IMG_PATH)
    if len(img.shape) <3:
      continue
    TEST.append(img)
    TEST_LABELS.append((np.reshape(ONE_HOT_ENCODED[MAP_DIC[class_id]], [200])))
  return np.array(TEST), np.array(TEST_LABELS)

In [20]:
from tensorflow import keras

TRAIN_IMAGES, TRAIN_LABELS = getImage('train')
TEST_IMAGES, TEST_LABELS = loadValidationData('val')


TEST_IMAGES = keras.applications.resnet50.preprocess_input(TEST_IMAGES)
TRAIN_IMAGES =  keras.applications.resnet50.preprocess_input(TRAIN_IMAGES)

print(TRAIN_IMAGES.shape)
print(TRAIN_LABELS.shape)

print(TEST_IMAGES.shape)
print(TEST_LABELS.shape)

(100000, 64, 64, 3)
(100000, 200)
(10000, 64, 64, 3)
(10000, 200)


In [21]:
!pip install hyperas

In [ ]:
from hyperas import optim

def model(x_train, y_train, x_test, y_test):
    """
    Model providing function:

    Create Keras model with double curly brackets dropped-in as needed.
    Return value has to be a valid python dictionary with two customary keys:
        - loss: Specify a numeric evaluation metric to be minimized
        - status: Just use STATUS_OK and see hyperopt documentation if not feasible
    The last one is optional, though recommended, namely:
        - model: specify the model just created so that we can later use it again.
    """
    input_shape = K.Input(shape=(64,64,3))

    model_mlp = Sequential()
    model_mlp.add(Dense({{choice([32, 64,126, 256, 512, 1024])}},
                        activation='relu', input_shape= input_shape))
    model_mlp.add(Dropout({{uniform(0, .5)}}))
    model_mlp.add(Dense({{choice([32, 64, 126, 256, 512, 1024])}}))
    model_mlp.add(Activation({{choice(['relu', 'sigmoid'])}}))
    model_mlp.add(Dropout({{uniform(0, .5)}}))
    model_mlp.add(Dense({{choice([32, 64, 126, 256, 512, 1024])}}))
    model_mlp.add(Activation({{choice(['relu', 'sigmoid'])}}))
    model_mlp.add(Dropout({{uniform(0, .5)}}))
    model_mlp.add(Dense({{choice([32, 64, 126, 256, 512, 1024])}}))
    model_mlp.add(Activation({{choice(['relu', 'sigmoid'])}}))
    model_mlp.add(Dropout({{uniform(0, .5)}}))
    model_mlp.add(Dense(200))
    model_mlp.add(Activation({{choice(['softmax','linear'])}}))
    model_mlp.compile(loss={{choice(['categorical_crossentropy','mse'])}}, metrics=['accuracy'],
                  optimizer={{choice(['rmsprop', 'adam', 'sgd'])}})


model = model()
model.fit(TRAIN_IMAGES, TRAIN_LABELS,
          batch_size={{choice([16, 32, 64, 128])}},
          epochs=2,
          verbose=2,
          validation_data=(x_test, y_test))
score, acc = model_mlp.evaluate(TEST_IMAGES, TEST_LABELS, verbose=1)
print('Test accuracy:', acc)
return {'loss': -acc, 'status': STATUS_OK, 'model': model_mlp}

In [ ]:
 best_run, best_model = optim.minimize(model=model,
                                              data=data,
                                              algo=tpe.suggest,
                                              max_evals=2,
                                              trials=Trials())
        X_train, Y_train, X_test, Y_test = data()
        print("Evalutation of best performing model:")
        print(best_model.evaluate(X_test, Y_test))
        print("Best performing model chosen hyper-parameters:")
        print(best_run)